In [ ]:
from src.assets import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import uncertainties as unc

In [ ]:
import mplhep as hep

In [ ]:
hep.style.use(hep.style.CMS)

In [ ]:
data_df = pd.read_csv('combined_xml_report.csv')

In [ ]:
def sign(x):
    if x==0: return 1
    return abs(x)/x

In [ ]:
def calc_chi2(x, var):
    return np.sum(x**2/var)
def calc_chi2_width(x, stddev, width):
    return calc_chi2(x, (stddev**2+width**2*sign(width)))

In [ ]:
class Chi2Curve:
    def __init__(self, steps, chi2, nDF):
        self.steps = steps
        self.chi2  = chi2
        self.nDF = nDF
    def step_size(self):
        #assume constant step size 
        return self.steps[1]-self.steps[0]
    def nChi2(self):
        return self.chi2/self.nDF
    def pdf(self):
        pdf = np.array(list(map( lambda x: stats.chi2.pdf(x, self.nDF), self.chi2)))
        pdf = np.nan_to_num(pdf)
        # total sums up to 1
        pdf = pdf/np.sum(pdf)
        total_at_start = self.cdf()[0]
        return pdf*total_at_start
    def mean_value(self):
        return np.sum(self.steps*self.pdf())/np.sum(self.pdf())
        return unc.ufloat(1,1)
    def cdf(self):
        return  np.array(list(map( lambda x: stats.chi2.cdf(x, self.nDF), self.chi2)))
    def cdf_range(self):
        cdf = self.cdf()
        one_sigma_bound = .16
        min_index, max_index = np.argmin(abs(cdf-(1-one_sigma_bound))), np.argmin(abs(cdf-one_sigma_bound))
        return self.steps[min_index], self.steps[max_index]

In [ ]:
def scan_offsets(x, unc, steps=np.linspace(0,.2,1000)):
    return steps, list(map(lambda width: calc_chi2_width(x, unc, width), steps))

In [ ]:
def scan_offset_df(df, alignable, steps):
    nDF = df.shape[0]
    x = df[alignable]
    unc = df[alignable+'_unc']
    return Chi2Curve(*np.array(scan_offsets(x, unc, steps=steps)), nDF)

In [ ]:
def draw_chi2_curve(ax, alignable, df, title, steps, unit, scale):
    chi2_curve = scan_offset_df(df, alignable, steps)
    mv = chi2_curve.mean_value()
    chi2_range = chi2_curve.cdf_range()
    #ax.plot(chi2_curve.steps*scale, chi2_curve.pdf(), label = '{}, {:.1f} {}'.format(title, mv*scale, unit))
    ax.plot(chi2_curve.steps*scale, chi2_curve.pdf(), label = '{}, {:.1f}-{:.1f} {}'.format(title, chi2_range[0]*scale, chi2_range[1]*scale, unit))
    ax.legend()
    ax.set_ylabel('probability density')
    ax.set_xlabel(unit)
    return mv

In [ ]:
#first collumn units in cms or radians last one is the conversion to intended units
x_units = [
    [.2, 'micrometers',10000],
    [.2, 'micrometers',10000],
    [.2, 'micrometers',10000],
    [.001, 'milliradians', 1000],
    [.001, 'milliradians', 1000],
    [.001, 'milliradians', 1000],
]

In [ ]:
def plot_station_alignable(sn, ax, alignable, steps, unit, scale):
    draw_chi2_curve(ax, alignable, data_df[data_df.station==sn], 'station {} {}'.format(sn, alignable), steps, unit, scale)

In [ ]:
chi2_curve = scan_offset_df(data_df[data_df.station==4], 'phiy', np.linspace(0,.001,1000))

In [ ]:
for al, (xmax, unit, scale) in zip(alignables, x_units):
    fig, ax = plt.subplots(figsize=(12,10))
    steps = np.linspace(0, xmax, 1000)
    for i in [1,2,3,4]:
        plot_station_alignable(i, ax, al, steps, unit, scale)
    #ax.plot([0,1], [1,1])
    fig.show()
    fig.savefig('chi2_output/pdf_{}_stations.pdf'.format(al))

In [ ]:
def plot_wheel_alignable(wheel, ax, alignable, steps, unit, scale):
    draw_chi2_curve(ax, alignable, data_df[abs(data_df.wheel)==wheel], 'wheel {} {}'.format(wheel, alignable), steps, unit, scale)

In [ ]:
for al, (xmax, unit, scale) in zip(alignables, x_units):
    fig, ax = plt.subplots(figsize=(6,5))
    steps = np.linspace(0, xmax, 1000)
    for i in [0,1,2]:
        plot_station_alignable(i, ax, al, steps, unit, scale)
    #ax.plot([0,1], [1,1])
    fig.show()
    fig.savefig('chi2_output/pdf_{}_wheel.pdf'.format(al))